In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from sklearn.metrics import mean_squared_error
import statsmodels.api as sm
from statsmodels.formula.api import ols

# Load the data
datadir = "J:/private/SYS4021/2021/Data/VAweather/"
sourcedir = "J:/private/SYS4021/2021/R_Code"
VAweather = pd.read_csv(f'{datadir}/VirginiaWeatherData.csv')

# Replace -999s with NAs
VAweather.replace(-999.0, np.nan, inplace=True)
VAweather = VAweather.iloc[:-7]  # Exclude the last 7 rows

# Create date column for plotting
VAweather['date'] = pd.to_datetime(VAweather[['Year', 'Month']].assign(DAY=1))

# Impute NAs using interpolation
VAweather[['C_Precip', 'C_Tmax', 'C_Tmin', 'R_Precip', 'R_Tmax', 'R_Tmin']] = VAweather[['C_Precip', 'C_Tmax', 'C_Tmin', 'R_Precip', 'R_Tmax', 'R_Tmin']].interpolate()

# Create time series of monthly Richmond precipitation and minimum temperature
Richmond_P = VAweather['R_Precip']
Richmond_Tmin = VAweather['R_Tmin']

# Use the pd.Series to get a time series of Richmond_P
precip_ts = pd.Series(Richmond_P.values)
temp_ts = pd.Series(Richmond_Tmin.values)

# Temp Linear Models: Trend and Seasonality
time_temp = np.arange(len(temp_ts)-6)
temp_trend_seasonal = ols("temp_ts[time_temp] ~ time_temp + np.sin(2*np.pi*time_temp/12) + np.cos(2*np.pi*time_temp/12)", data=VAweather).fit()

# Summary of the linear model
print(temp_trend_seasonal.summary())

# Plot the true vs. fitted time series
plt.figure(figsize=(10, 6))
plt.plot(VAweather['date'][:len(temp_ts)-6], temp_ts[:len(temp_ts)-6], label='True')
plt.plot(VAweather['date'][:len(temp_ts)-6], temp_trend_seasonal.fittedvalues, color='red', label='Fitted')
plt.xlabel('Date')
plt.ylabel('Richmond Tmin')
plt.legend()
plt.show()

# AR, MA & ARIMA Models
e_ts_temp = temp_ts[time_temp] - temp_trend_seasonal.fittedvalues

# Plot the residuals
plt.figure(figsize=(10, 6))
plt.plot(e_ts_temp)
plt.xlabel('Time')
plt.ylabel('Residuals from Temperature Model')
plt.show()

# Plot the ACF and PACF of the residuals
fig, axes = plt.subplots(2, 1, figsize=(12, 8))
plot_acf(e_ts_temp, ax=axes[0])
plot_pacf(e_ts_temp, ax=axes[1])
plt.show()

# Choose p and q terms for e_ts_temp based on the acf and pacf
# ar(1) p=1
temp_ar1 = SARIMAX(e_ts_temp, order=(1, 0, 0), trend='n').fit()
print(temp_ar1.summary())

# ma(2) p=0, q=2
temp_ma2 = SARIMAX(e_ts_temp, order=(0, 0, 2), trend='n').fit()
print(temp_ma2.summary())

# arma(1,2) p=1, q=2
temp_arma12 = SARIMAX(e_ts_temp, order=(1, 0, 2), trend='n').fit()
print(temp_arma12.summary())

# Based on AIC, which one do you choose?
temp_auto = sm.tsa.statespace.SARIMAX(e_ts_temp, order=(2, 0, 1)).fit()
print(temp_auto.summary())

# BIC
print(temp_ar1.bic, temp_ma2.bic, temp_arma12.bic, temp_auto.bic)

# Assess residuals vs. fitted
fig, axes = plt.subplots(2, 2, figsize=(12, 12))
sns.scatterplot(x=temp_ar1.fittedvalues, y=temp_ar1.resid, ax=axes[0, 0]).set_title('AR1')
sns.scatterplot(x=temp_ma2.fittedvalues, y=temp_ma2.resid, ax=axes[0, 1]).set_title('MA2')
sns.scatterplot(x=temp_arma12.fittedvalues, y=temp_arma12.resid, ax=axes[1, 0]).set_title('ARMA12')
sns.scatterplot(x=temp_auto.fittedvalues, y=temp_auto.resid, ax=axes[1, 1]).set_title('Auto')
plt.show()

# Assess normality of residuals
fig, axes = plt.subplots(2, 2, figsize=(12, 12))
sm.qqplot(temp_ar1.resid, line='s', ax=axes[0, 0]).set_title('AR1')
sm.qqplot(temp_ma2.resid, line='s', ax=axes[0, 1]).set_title('MA2')
sm.qqplot(temp_arma12.resid, line='s', ax=axes[1, 0]).set_title('ARMA12')
sm.qqplot(temp_auto.resid, line='s', ax=axes[1, 1]).set_title('Auto')
plt.show()

# Diagnostics for independence of residuals using tsdiag()
fig, ax = plt.subplots(figsize=(10, 8))
sm.graphics.tsa.plot_acf(temp_auto.resid, ax=ax)
plt.show()

# Plot the autocorrelation (ACF) and partial autocorrelation (PACF) of the residuals of temp.auto
fig, axes = plt.subplots(2, 1, figsize=(12, 8))
plot_acf(temp_auto.resid, ax=axes[0])
plot_pacf(temp_auto.resid, ax=axes[1])
plt.show()

# Plot the best model's fitted values vs. true values
Tmin_fitted = temp_trend_seasonal.fittedvalues + temp_auto.fittedvalues
plt.figure(figsize=(10, 6))
plt.plot(temp_ts[:len(time_temp)], label='True')
plt.plot(Tmin_fitted, color='red', label='Fitted')
plt.xlabel('Time')
plt.ylabel('Richmond Tmin')
plt.legend()
plt.show()

# Forecast with the best model:
temp_auto_forecast = temp_auto.get_forecast(steps=6)
temp_auto_forecast_ci = temp_auto_forecast.conf_int()

# Prediction performance
next_6mo_time = np.arange(len(temp_ts)-6, len(temp_ts))
next_6mo = pd.DataFrame({'time_temp': next_6mo_time, 'temp': temp_ts[next_6mo_time]})

E_Y_pred = temp_trend_seasonal.predict(new_data=next_6mo)
e_t_pred = temp_auto_forecast.predicted_mean
next_6mo_prediction = E_Y_pred + e_t_pred

# MSE:
mse = mean_squared_error(next_6mo['temp'], next_6mo_prediction)
print(f'MSE: {mse}')

# Plot actual values and predicted values
plt.figure(figsize=(10, 6))
plt.plot(next_6mo['temp'], 'o-', label='Actual')
plt.plot(next_6mo_prediction, 'o-', color='red', label='Predicted')
plt.fill_between(np.arange(6), temp_auto_forecast_ci.iloc[:, 0], temp_auto_forecast_ci.iloc[:, 1], color='red', alpha=0.3)
plt.xlabel('Time')
plt.ylabel('Richmond Tmin')
plt.legend()
plt.show()

# Simulate 50 years of monthly minimum temperatures with the best model
np.random.seed(1)
temp_resid_sim = sm.tsa.arma_generate_sample(ar=[1, -temp_auto.params['ar.L1'], -temp_auto.params['ar.L2']], 
                                             ma=[temp_auto.params['ma.L1']], 
                                             nsample=12*50, 
                                             scale=np.sqrt(temp_auto.sigma2))

# Add mean predictions and plot simulation of Tmin
time_50yr = np.arange(1, 50*12+1)
temp_50_yr_mean = temp_trend_seasonal.predict(new_data=pd.DataFrame({'time_temp': time_50yr}))

# Plot simulated temperatures
plt.figure(figsize=(12, 6))
plt.plot(temp_50_yr_mean + temp_resid_sim)
plt.xlabel('Time')
plt.ylabel('Simulated Richmond Tmin')
plt.show()

# Repeat the process for precip
# Model Trend and Seasonality
# Similar steps will be taken for precipitation data...
